# REI505M Machine Learning - Final project
### Due: --------

**Names**: Axel Kristján Axelsson, Bjarni Haukur Bjarnason <br />
**Email**: aka30@hi.is, bhb23@hi.is

**Part 1**

Take a close look at the recurrent neural network code shown in class (`CodeExamples-10` on Canvas). The code demonstrates how sequence to sequence learning can be used to learn how to add together small numbers. This is done by presenting the network with input-output pairs, where the input is a string on the form "123+456" and the output is "579".

Modify the code for addition so that the network learns *multiplication* of two digit numbers (Exerccise 1.1, Exercise 1.2 and Exercise 1.3). You may need to modify the number of hidden nodes and layers in the network, as well as the number of epochs. You should not expect to get a network that predicts all the examples correctly.

When you have finished training your network, report the training and validation accuracy returned by Keras, as well as the the fraction of correctly classified validation examples (Exercise 1.4 and Exercise 1.5). Provide a graph of the training and validation accuracy as a function of epoch number. 

You should mark all the modifications that you do to the code by inserting comments on the form
```python
# START MODIFICATION
...
# END MODIFICATION
```

*Note*: It is harder for the RNN to learn multiplication than addition and you therefore need to increase the size of the training set considerably. The code generates training examples at random, rejecting those that already exist. This results in long running times when the training size becomes large (a more efficient strategy would be to generate all possible training examples and seleecting a subset at random).